In [1]:
import pandas as pd
import numpy as np
import librosa
import h5py

In [73]:
one_to_loop_ratio = 1
loop_unit_multi_sample_pct = 0.2

In [11]:
meta_data = pd.read_csv('audio_metadata - filtered.csv', sep=',')
# meta_data.drop(columns=['group', 'category', 'sub_category'], inplace=True)

In [12]:
df_id = np.array(meta_data['id'].astype(int)).flatten()

wavs = []
srs = []

def load_sample(f, sample_id):
    group = f["audio_data"][str(sample_id)]
    waveform = np.array(group["waveform"])
    sample_rate = np.array(group["sample_rate"])
    return waveform, sample_rate


with h5py.File('filtered_audio_data.h5', "r") as f:
    for i in df_id:
        waveform, sr = load_sample(f, i)  # load sample with id=42
        wavs.append(waveform)
        srs.append(sr)


samples = pd.DataFrame({'id': df_id, 'waveform': wavs, 'sample_rate': srs, })

In [ ]:
samples = samples.set_index('id').join(meta_data.set_index('id'))
one_shots = samples.copy()
loops = samples.copy()

In [70]:
if 'one_shot_intent' and 'loop_id' in samples.columns:
    print("'loop_id' and 'one_shot_intent' are present")
    one_shots = one_shots[one_shots['One_Shot'] == 1 and one_shots['one_shot_intent']==1]
    loops = loops[loops['loop_id'] > 0]
else:
    print("'loop_id' and 'one_shot_intent' are not present")
    one_shots = one_shots[one_shots['One_Shot'] == 1]
    loops = loops[loops['One_Shot'] == 0]

'loop_id' and 'one_shot_intent' are not present


In [65]:
one_shot_cats = set(one_shots['category'])
loop_cats = set(one_shots['category'])

In [77]:
assert one_shot_cats == loop_cats
cats = {}
for cat in one_shot_cats:
    cats[cat] = {}
    cats[cat]["current_loop"] = len(loops[loops['category'] == cat])
    cats[cat]["current_one_shot"] = len(one_shots[one_shots['category'] == cat])
    cats[cat]["all"] = cats[cat]["current_loop"] + cats[cat]["current_one_shot"]
    cats[cat]['needed_each'] = int((one_to_loop_ratio * cats[cat]["all"]) / (one_to_loop_ratio +1 + loop_unit_multi_sample_pct))


cats

{'Hi Hat': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545},
 'Kick': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545},
 'Clap': {'current_loop': 195,
  'current_one_shot': 1005,
  'all': 1200,
  'needed_each': 545}}

# next up  
made a way to import files. get their category and count by one shot or loop  

now i need to do the following:  
- figure out the number of samples to switch from one shot to loop in order to have a 50/50 split  
  - a small amount (ex: 20%) of loops should use more than 1 sample. need to take into account that we'll use a higher amount of samples for loops than oneshots to acheive a 50/50 split: done  
- start choosing the one shots to use in a loop  
- doccument their intent
- all of this needs to work with the possibility that loops may or may not exist. can't overwrite existing data when making loops. loop ids can't clash with existing loop ids



# line to save for when 
keep this line for when i need to start generating new ids for the loops
loop_id_num = max(loops['loop_id'])